# A1: core module notebook

# Introduction

This notebook includes a description of the 'core' python module in the JBEI Quantitative Metabolic Modeling (QMM) library. A description and demonstration of the diffent classes can be found below. 

# Setup

First, we need to set the path and environment variable properly:

In [1]:
quantmodelDir = '/users/hgmartin/libraries/quantmodel'

This is the only place where the jQMM library path needs to be set.

In [2]:
%matplotlib inline

import sys, os
pythonPath = quantmodelDir+"/code/core"
if pythonPath not in sys.path:
    sys.path.append(pythonPath)
os.environ["QUANTMODELPATH"] = quantmodelDir

Importing the required modules for the demo:

In [3]:
from IPython.display import Image
import core, FluxModels
import os

# Classes description

## Metabolite related classes

### metabolite class

The *metabolite* class is used to store all information related to a metabolite. For example the following instantation:

In [4]:
ala = core.Metabolite('ala-L', ncarbons=3, source=True, feed='100% 1-C', destination=False, formula='C3H7NO2')

creates a metabolite with nbame 'ala-L', 3 carbon atoms, which is the source of labeling, is labeled in the first carbon, 
is not a destination (measured) metabolite and with a composition formula equal to 'C3H7NO2'

the **generateEMU** function creates the corresponding Elementary Metabolite Unit (EMU):

In [5]:
ala.generateEMU([2])

'ala-L_1_3'

In this case the EMU contains the first and last carbon in alanine. The input ([2]) specifies which carbons to exclude:

In [6]:
ala.generateEMU([2,3])

'ala-L_1'

### reactant and product classes

*Reactant* and *Product* are classes derived from metabolite and the only difference is that they represent metabolites in the context of a reaction. Hence, the stoichiometry of the metabolite and the labeling pattern in that reaction are included: 

In [7]:
R_ala = core.Reactant(ala, 1, 'abc')

Notice that the stoichiometry information (1, meaning in the reaction only 1 molecule participates in the reaction) and the labeling data ('abc', one part of the labeling pattern, see below) only make sense in the context of a reaction, so they are not included in the metabolite class.

Both classes are derived from metabolites, so they inherit their methods:

In [8]:
R_ala.generateEMU([2,3])

'ala-L_1'

## Reaction related classes

### reaction class

The *Reaction* class produces a reaction instance:

In [9]:
# Create reactant metabolites
coa_c = core.Metabolite('coa_c')
nad_c = core.Metabolite('nad_c')
pyr_c = core.Metabolite('pyr_c')

# Convert into reactants
Rcoa_c = core.Reactant(coa_c, 1.0)
Rnad_c = core.Reactant(nad_c, 1.0)
Rpyr_c = core.Reactant(pyr_c, 1.0)

# Create product metabolites
accoa_c = core.Metabolite('accoa_c')
co2_c   = core.Metabolite('co2_c')
nadh_c  = core.Metabolite('nadh_c')

# Convert into products
Raccoa_c = core.Product(accoa_c, 1.0)
Rco2_c   = core.Product(co2_c, 1.0)
Rnadh_c  = core.Product(nadh_c, 1.0)

# Create reaction
PDH = core.Reaction('PDH',reactants=[Rcoa_c,Rnad_c,Rpyr_c] , products=[Raccoa_c,Rco2_c,Rnadh_c] 
                    ,subsystem='S_GlycolysisGluconeogenesis')

Reactions can also initialized from a string:

In [10]:
PDH2 = core.Reaction.from_string('PDH : coa_c + nad_c + pyr_c --> accoa_c  + co2_c   + nadh_c  ')

The *reaction* class contains some useful functions such as:

**stoichLine** to obtain the stoichiometric line for the reaction:

In [11]:
print PDH.stoichLine()
print PDH2.stoichLine()

PDH : coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
PDH : coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c


**getReactDict** produces a dictionary of reactants:

In [12]:
PDH.getReactDict()

{'coa_c': <core.Reactant instance at 0x7f985810a368>,
 'nad_c': <core.Reactant instance at 0x7f985810a3b0>,
 'pyr_c': <core.Reactant instance at 0x7f985810a3f8>}

**getProdDict** produces a dictionary of products:

In [13]:
PDH.getProdDict()

{'accoa_c': <core.Product instance at 0x7f985810a518>,
 'co2_c': <core.Product instance at 0x7f985810a560>,
 'nadh_c': <core.Product instance at 0x7f985810a5a8>}

## Elementary Metabolite Unit (EMU) related classes

Elementary Metabolite Units (or EMUs) of a compound are the molecule parts (moieties) comprising any distinct subset of the compound’s atoms (Antoniewicz MR, Kelleher JK, Stephanopoulos G: Elementary metabolite units (EMU): a novel framework for modeling isotopic distributions. Metab Eng 2007, 9:68-86.). For example, cit$_{123}$ represents the first 3 carbon atoms in the citrate molecule.

### EMU class

The *EMU* class provides a class to hold and manipulate emus:

In [14]:
cit321= core.EMU('cit_3_2_1')

The method **findnCarbons** produces the number of carbons in the emu:

In [15]:
print cit321.findnCarbons()

3.0


The method **getMetName** produces the name of the corresponding metabolite:

In [16]:
print cit321.getMetName()

cit


In [17]:
str(cit321.getMetName())=='cit'

True

The method **getIndices** produces the indices:

In [18]:
print cit321.getIndices()

[3, 2, 1]


**getSortedName** sorts the indices in the emu name:

In [19]:
print cit321.getSortedName()

cit_1_2_3


**getEmuInSBML** produces the name of the emu in SBML format:

In [20]:
print cit321.getEmuInSBML()

cit_c_3_2_1


## Transitions related classes

Transitions contain the information on how carbon (or other) atoms are passed in each reaction. Atom transitions describe, for example, the fate of each carbon in a reaction, whereas emu transitions describe this information by using EMUs, as described below. 


### atomTransition class

Atom transitions represent the fate of each carbon in a reaction (Wiechert W. (2001) 13C metabolic flux analysis. Metabolic engineering 3: 195-206). For example, in:

AKGDH	akg --> succoa + co2	abcde : bcde + a

akg gets split into succoa and co2, with the first 4 carbons going to succoa and the remaining carbon going to co2. 


In [21]:
AT = core.AtomTransition('AKGDH	akg --> succoa + co2	abcde : bcde + a')
print AT

AKGDH	akg --> succoa + co2	abcde : bcde + a


The method **findEMUtransition** provides for a given input emu (e.g. succoa_1_2_3_4), which emu it comes from in the form of a emu transition:

In [22]:
emu1 = core.EMU('co2_1')
print AT.findEMUtransition(emu1)

emu2 = core.EMU('succoa_1_2_3_4')
print AT.findEMUtransition(emu2)

['AKGDH, akg_1 --> co2_1']
['AKGDH, akg_2_3_4_5 --> succoa_1_2_3_4']


This is done through the method **findEMUs**, which finds the emus from which the input emanates in the given atom transition:

In [23]:
print emu2.name
print AT.findEMUs(emu2)
for emus in AT.findEMUs(emu2):
    for emu_ in emus:
        print emu_.name

succoa_1_2_3_4
[[<core.EMU instance at 0x7f985810ed88>]]
akg_2_3_4_5


which in turn, uses the method **getOriginDictionary** which provides for a given input emu the originating metabolite and the correspondance in indices: 

In [24]:
AT.getOriginDictionary(emu2)

{'akg': (['b', 'c', 'd', 'e'], [2, 3, 4, 5])}

### EMUtransition class

Class for EMU transitions that contain information on how different EMUs transform intto each other. For example:
    
        TA1_b,  TAC3_c_1_2_3 + g3p_c_1_2_3 -->  f6p_c_1_2_3_4_5_6
        
indicating that TAC3_c_1_2_3 and g3p_c_1_2_3 combine to produce f6p_c_1_2_3_4_5_6 in reaction TA1_b (backward reaction of TA1), or:
        
        SSALy, (0.5) sucsal_c_4 --> (0.5) succ_c_4
        
which indicates that the fourth atom of sucsal_c becomes the fourth atom of succ_c. The (0.5) contribution coefficient indicates that reaction SSALy contains a symmetric molecule and two labeling correspondences are equally likely. Hence this transition only contributes half the flux to the final labeling.

In [25]:
emuTrans = core.EMUTransition('TA1_b,  TAC3_c_1_2_3 + g3p_c_1_2_3 -->  f6p_c_1_2_3_4_5_6')
print emuTrans

TA1_b, TAC3_c_1_2_3 + g3p_c_1_2_3 --> f6p_c_1_2_3_4_5_6


In [26]:
str(emuTrans) == 'TA1_b, TAC3_c_1_2_3 + g3p_c_1_2_3 --> f6p_c_1_2_3_4_5_6'

True

## Ranged number class

The *rangedNumber* class describes floating point numbers for which a confidence interval is available. For example, fluxes obtained through 2S-$^{13}$C MFA are described through the flux that best fits the data and the highest and lowest values that are found to be compatible with labeling data (see equations 16-23 in Garcia Martin *et al* 2015). However, this class has been abstracted out so it can be used with other ranged intervals. Ranged numbers can used as follows:

In [27]:
number = core.rangedNumber(0.3,0.6,0.9) # 0.3 lowest, 0.6 best fit, 0.9 highest

Ranged numbers can be printed:
    

In [28]:
print number

[0.3:0.6:0.9]


and added, substracted, multiplied and divided following the standard error propagation rules (https://en.wikipedia.org/wiki/Propagation_of_uncertainty):

In [29]:
A = core.rangedNumber(0.3,0.6,0.9)
B = core.rangedNumber(0.1,0.15,0.18)

In [30]:
print A+B

[0.445861873485:0.75:1.05149626863]


In [31]:
print A-B

[0.145861873485:0.45:0.751496268634]


In [32]:
print 2*A

[0.6:1.2:1.8]


In [33]:
print B/3

[0.0333333333333:0.05:0.06]


## Flux class

The flux class describes fluxes attached to a reaction. For example, if the net flux is described by the ranged number A and the exchange flux by the ranged number B, the corresponding flux would be:

In [34]:
netFlux      = A
exchangeFlux = B
flux1 = core.flux(net_exc_tup=(netFlux,exchangeFlux)) 

In [35]:
print flux1

Forward: [0.445861873485:0.75:1.05149626863]
Backward: [0.1:0.15:0.18]
Net: [0.3:0.6:0.9]
Exchange: [0.1:0.15:0.18]



Fluxes can be easily multiplied:

In [36]:
print 3*flux1

Forward: [1.33758562046:2.25:3.1544888059]
Backward: [0.3:0.45:0.54]
Net: [0.9:1.8:2.7]
Exchange: [0.3:0.45:0.54]

